In [1]:
import xlrd
import pandas as pd
import numpy as np
import json
import unidecode

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_excel('input/youth_survey_raw.xlsx')

# get question ids from columns
question_ids = [ c for c in df.columns if 'Q' in c ]

# copy row 0 to column names
df.columns = df.iloc[0]

# delete row 0
df = df.drop(0, axis=0)
df = df.applymap(str)


,M1: Questionnaire No,Project No,Project Name,M2: Governorate,M3: District,M7: Urbanity,M8: Date of interview,M8a: Day of interview,M9: Interview Validation,M11b: Date of Auditing,...,Q61_3_X: Other,S1: How old are you,S2: What is your marital status?,S3: Do you have any children?,"S4: (If ""yes""), How many?","S5: Including you, how many people, children, above and under 15 live in this household?",S6: What is the highest education level you completed?,"S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?",S8: What is your job status today?,S11: Gender of respondent
1,79,132,Youth Survey,Ibb,Al-Sabrah,Rural,2017-05-11 00:00:00,Thu,Not validated,2017-05-11 00:00:00,...,nan,17,"Single, didn't not get married before",nan,nan,6,Didn't complete secondary school,"20,000 YER- 39,000 YER",Student,Male
2,101,132,Youth Survey,Ibb,Al-Makhader,Rural,2017-05-12 00:00:00,Fri,Accompanied interview,2017-05-12 00:00:00,...,nan,16,"Single, didn't not get married before",nan,nan,6,Completed intermediate school,"20,000 YER- 39,000 YER",Student,Male
3,172,132,Youth Survey,Abyan,Khanfar,Rural,2017-05-10 00:00:00,Wed,Re-contacted by person,2017-05-10 00:00:00,...,nan,25,Married,Yes,4,6,Illiterate (Can't read and write),"Less than 20,000 YER",Housewife,Female
4,1107,132,Youth Survey,Shabwah,Jardaan,Rural,2017-05-23 00:00:00,Tue,Not validated,2017-05-23 00:00:00,...,nan,19,Married,No,nan,9,Completed secondary school,"60,000 YER - 79,000 YER",Housewife,Female
5,1439,132,Youth Survey,ad-Dali,Qatabah,Rural,2017-05-10 00:00:00,Wed,Not validated,2017-05-10 00:00:00,...,nan,18,"Single, didn't not get married before",nan,nan,13,Didn't complete secondary school,"60,000 YER - 79,000 YER",Student,Male
6,917,132,Youth Survey,Hadhramawt,Al-Mukala City,Urban,2017-05-14 00:00:00,Sun,Not validated,2017-05-14 00:00:00,...,nan,16,"Single, didn't not get married before",nan,nan,4,Didn't complete secondary school,(DK),Student,Female
7,170,132,Youth Survey,Ibb,Yarim,Rural,2017-05-17 00:00:00,Wed,Not validated,2017-05-17 00:00:00,...,nan,24,Married,Yes,1,3,Completed elementary school,"20,000 YER- 39,000 YER",Housewife,Female
8,74,132,Youth Survey,Ibb,Al-Sabrah,Rural,2017-05-11 00:00:00,Thu,Not validated,2017-05-11 00:00:00,...,nan,25,Divorced,Yes,4,5,Illiterate (Can't read and write),"20,000 YER- 39,000 YER",Works in his/her own business,Female
9,1462,132,Youth Survey,ad-Dali,Al-Dhale,Urban,2017-05-14 00:00:00,Sun,Not validated,2017-05-14 00:00:00,...,nan,16,"Single, didn't not get married before",nan,nan,5,Didn't complete secondary school,"Less than 20,000 YER",Student,Male
10,1436,132,Youth Survey,ad-Dali,Qatabah,Rural,2017-05-10 00:00:00,Wed,Re-contacted by person,2017-05-10 00:00:00,...,nan,20,"Single, didn't not get married before",nan,nan,8,Completed secondary school,"20,000 YER- 39,000 YER","Unemployed, looking for job",Male


In [66]:
question_columns = [col.strip() for col in df.columns if col.startswith(('Q'))]
demographic_columns = [col.strip() for col in df.columns if col.startswith(('S'))]
demographic_columns

['S1: How old are you',
 'S2: What is your marital status?',
 'S3: Do you have any children?',
 'S4: (If "yes"), How many?',
 'S5: Including you, how many people, children, above and under 15 live in this household?',
 'S6: What is the highest education level you completed?',
 'S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?',
 'S8: What is your job status today?',
 'S11: Gender of respondent']

In [192]:
df_ = df.filter(items=question_columns)
df_['governorate'] = df['M2: Governorate']
df_['district'] = df['M3: District']
df_['urbanity'] = df['M7: Urbanity']
df_['household_monthly_income'] = df['S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?']
df_['age'] = df['S1: How old are you']
df_['marital_status'] = df['S2: What is your marital status?'].apply(lambda x: x.split(', ')[0])
df_['have_children'] = df['S3: Do you have any children?']
df_['job_status'] = df['S8: What is your job status today?'].apply(lambda x: x.replace('Unemployed, looking for job', 'Unemployed').replace('Works in his/her own business', 'Self-employed'))
df_['gender'] = df['S11: Gender of respondent']

df_.head()

,Q1: First I would like to ask you about your current situation / family situation. How would you rate it on a scale from 1 to 5?,"Q2: In general, what do you think about the current situation in Yemen, is it going in the right direction or in the wrong direction?",Q3: What is your status:,Q3x: Other,"Q4_1: What are the reasons you had to stop going to school, university, vocational training or lost your job? First answer",Q4_1_X :Other,"Q4_2: What are the reasons you had to stop going to school, university, vocational training or lost your job? Second answer",Q4_2_X :Other,Q5: How much time do you spend at school/university/work/vocational training?,Q6: How close is the school/university/work/vocational training you attend to your house?,...,Q61_3_X: Other,governorate,district,urbanity,household_monthly_income,age,marital_status,have_children,job_status,gender
1,Neither bad nor good,In a very bad direction,School student,nan,nan,nan,nan,nan,5-8 hours,0-1 km,...,nan,Ibb,Al-Sabrah,Rural,"20,000 YER- 39,000 YER",17,Single,nan,Student,Male
2,It’s very bad,"3.Not in the right direction, not in the wrong...",School student,nan,nan,nan,nan,nan,1-4 hours,0-1 km,...,nan,Ibb,Al-Makhader,Rural,"20,000 YER- 39,000 YER",16,Single,nan,Student,Male
3,It’s very bad,In a very bad direction,I am a housewife,nan,nan,nan,nan,nan,nan,nan,...,nan,Abyan,Khanfar,Rural,"Less than 20,000 YER",25,Married,Yes,Housewife,Female
4,Neither bad nor good,"3.Not in the right direction, not in the wrong...",I am a housewife,nan,nan,nan,nan,nan,nan,nan,...,nan,Shabwah,Jardaan,Rural,"60,000 YER - 79,000 YER",19,Married,No,Housewife,Female
5,It’s somewhat good,In a very bad direction,School student,nan,nan,nan,nan,nan,1-4 hours,0-1 km,...,nan,ad-Dali,Qatabah,Rural,"60,000 YER - 79,000 YER",18,Single,nan,Student,Male


In [245]:
df_a_list = []
for q in question_columns:
    df_a = pd.DataFrame()
    
    
    
    q_ = unidecode.unidecode(q)
    qID = q_.split(':')[0]
    q_ = q_.split(':')[1].strip()
    
    
  
    if q_ != 'Other':
        df_a['governorate'] = df_['governorate']
        df_a['district'] = df_['district']
        df_a['qID'] = qID
        df_a['question_raw'] = q_
        df_a['answer_raw'] = df_[q]
        df_a['urbanity'] = df_['urbanity']
        df_a['household_monthly_income'] = df_['household_monthly_income']
        df_a['age'] = df_['age']
        df_a['marital_status'] = df_['marital_status']
        df_a['have_children'] = df_['have_children']
        df_a['job_status'] = df_['job_status']
        df_a['gender'] = df_['gender']
    
    
    

    df_a_list.append(df_a)

df_formatted = pd.concat(df_a_list)


df_formatted.head()

    

,governorate,district,qID,question_raw,answer_raw,urbanity,household_monthly_income,age,marital_status,have_children,job_status,gender
1,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"20,000 YER- 39,000 YER",17,Single,nan,Student,Male
2,Ibb,Al-Makhader,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"20,000 YER- 39,000 YER",16,Single,nan,Student,Male
3,Abyan,Khanfar,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"Less than 20,000 YER",25,Married,Yes,Housewife,Female
4,Shabwah,Jardaan,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"60,000 YER - 79,000 YER",19,Married,No,Housewife,Female
5,ad-Dali,Qatabah,Q1,First I would like to ask you about your curre...,It’s somewhat good,Rural,"60,000 YER - 79,000 YER",18,Single,nan,Student,Male


In [263]:
# df['avg_result'] = df.groupby(['a','b'])['result'].transform('mean')

df_formatted['answer_count'] = df_formatted.groupby(['governorate', 'question_raw', 'answer_raw']).transform('count')
df_formatted['answer_total'] = df_formatted.groupby(['governorate', 'question_raw']).transform('count')
df_formatted['answer_pct'] = (df_formatted['answer_count'] / df_formatted['answer_total'])*100

df_formatted
# df_formatted.sort_values(['district', 'qID'])

,governorate,district,qID,question_raw,answer_raw,urbanity,household_monthly_income,age,marital_status,have_children,job_status,gender,answer_count,answer_total
1,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"20,000 YER- 39,000 YER",17,Single,nan,Student,Male,61,170
2,Ibb,Al-Makhader,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"20,000 YER- 39,000 YER",16,Single,nan,Student,Male,53,170
3,Abyan,Khanfar,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"Less than 20,000 YER",25,Married,Yes,Housewife,Female,13,30
4,Shabwah,Jardaan,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"60,000 YER - 79,000 YER",19,Married,No,Housewife,Female,16,40
5,ad-Dali,Qatabah,Q1,First I would like to ask you about your curre...,It’s somewhat good,Rural,"60,000 YER - 79,000 YER",18,Single,nan,Student,Male,9,40
6,Hadhramawt,Al-Mukala City,Q1,First I would like to ask you about your curre...,Neither bad nor good,Urban,(DK),16,Single,nan,Student,Female,17,80
7,Ibb,Yarim,Q1,First I would like to ask you about your curre...,It’s somewhat bad,Rural,"20,000 YER- 39,000 YER",24,Married,Yes,Housewife,Female,46,170
8,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,It’s somewhat bad,Rural,"20,000 YER- 39,000 YER",25,Divorced,Yes,Self-employed,Female,46,170
9,ad-Dali,Al-Dhale,Q1,First I would like to ask you about your curre...,It’s very bad,Urban,"Less than 20,000 YER",16,Single,nan,Student,Male,7,40
10,ad-Dali,Qatabah,Q1,First I would like to ask you about your curre...,It’s somewhat good,Rural,"20,000 YER- 39,000 YER",20,Single,nan,Unemployed,Male,9,40


In [197]:
xls = pd.ExcelFile('tagged.xlsx')
tag_questions = pd.read_excel(xls, 'tag_questions')
rewrite_questions = pd.read_excel(xls, 'rewrite_questions')
rank_and_rewrite_answers = pd.read_excel(xls, 'rank_and_rewrite_answers')


# questions = tag_questions.merge(rewrite_questions, on = ['question_raw', 'tag'],how = 'outer')
# questions


# with_tagged_questions = questions.merge(df_formatted, on = ['question_raw'], how = 'outer')


# with_new_answers = rank_and_rewrite_answers.merge(with_tagged_questions, on = ['question_raw', 'answer_raw'], how = 'outer')

# with_new_answers = with_new_answers[['question_raw', 'question_en', 'answer_raw', 'answer_en', 'rank', 'governorate', 'district', 'urbanity', 'gender', 'marital_status', 'job_status']]
# with_new_answers

,qID_x,question_raw,tag,sub_tag,qID_y,question_en,question_ar,keep?
0,q1,First I would like to ask you about your curre...,safety,NaN,q1,How would you rate you/your family's current s...,NaN,y
1,q2,"In general, what do you think about the curren...",safety,NaN,q2,Is the situation in Yemen heading in the right...,NaN,y
2,q3,What is your status,daily life,NaN,q3,What is your employment status?,NaN,y
3,q4,What are the reasons you had to stop going to ...,education,NaN,q4,"Why did you stop going to school, university, ...",NaN,n
4,q5,What are the reasons you had to stop going to ...,education,NaN,q5,"Why did you stop going to school, university, ...",NaN,n
5,q6,How much time do you spend at school/universit...,education,NaN,q6,"How much time per day do you spend at school, ...",NaN,y
6,q7,How close is the school/university/work/vocati...,education,NaN,q7,"How close to your house is the school, univers...",NaN,y
7,q8,How do you get to your school/university/work/...,education,NaN,q8,"How do you get to your school, university, wor...",NaN,y
8,q9,In what physical condition is your school/univ...,education,NaN,q9,"What physical condition is your school, univer...",NaN,y
9,q10,"On a scale ranging from 1 to 5 in which ""1"" me...",education,NaN,q10,"How valued are you by your teachers, bosses, a...",NaN,y
